In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import json
from ControlDB import DB_CONNECTOR
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm

In [27]:
# MySQL 데이터베이스 연결 설정
db_config = {
    "host": "localhost",
    "user": "root",  # 본인의 MySQL 사용자 이름으로 변경
    "password": "wjdtkfkd12",  # 본인의 MySQL 비밀번호로 변경
    "database": "jiggle",
    "port": 3306,
    "auth_plugin": "mysql_native_password"
}

Connector = DB_CONNECTOR(db_config)
# read_sql = "SELECT * FROM Articles"

# df = Connector.read_data(read_sql)

In [3]:
df.head()

,id,placeName,address,link,comment,rate,platform
0,13,대충유원지,서울 마포구 월드컵북로6길 37 1층,https://brunch.co.kr/@archive-zeze/6,같은 곳 || 건축을 중심으로,3,브런치
1,14,대충유원지,서울 마포구 월드컵북로6길 37 1층,https://brunch.co.kr/@itrv/24,같은 곳 || 디자인을 중심으로,4,브런치
2,15,대충유원지,서울 마포구 월드컵북로6길 37 1층,https://m.sedaily.com/NewsView/1RY5VQGJQN/GK02...,같은 곳 || 기사,2,서울경제
3,16,스페이스깨,서울특별시 종로구 자하문로5길 15 스페이스깨 1층,https://fhhhfriends.com/%EC%8A%A4%ED%8E%98%EC%...,건축 스튜디오에서 작성,2,푸하하프렌즈
4,17,하우스서울,서울 송파구 백제고분로9길 5 1층,https://brunch.co.kr/@ratm820309n85i/292,no data,2,브런치


In [4]:
# 크롤링 관련 코드
def get_metadata(soup):
    meta_data = {}
    for meta in soup.find_all('meta'):
        if meta.get('name'):
            meta_data[meta.get('name')] = meta.get('content')
        elif meta.get('property'):
            meta_data[meta.get('property')] = meta.get('content')
    return meta_data

def get_text(soup):
    paragraphs = soup.find_all('p')
    text = '\n'.join([para.get_text() for para in paragraphs])
    return text

def get_images(soup):
    images = [img['src'] for img in soup.find_all('img') if 'src' in img.attrs]
    return images

In [22]:
# implement
links = df['link'].tolist()
data = []

chrome_driver_path = './chromedriver-win64/chromedriver-win64/chromedriver.exe'

# 셀레니움 설정
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

for index, link in tqdm(enumerate(links), desc="Processing links"):
    try:
        response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
        else:
            driver.get(link)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
        
        meta_data = get_metadata(soup)
        text = get_text(soup)
        images = get_images(soup)
        
        data.append({
            'P_id': df['id'].iloc[index],  # df의 id를 p_id로 추가
            'link': link,
            'meta_data': json.dumps(meta_data),  # 딕셔너리를 JSON 문자열로 변환
            'text': text,
            'images': json.dumps(images)  # 이미지 리스트를 JSON 문자열로 변환
        })
    except Exception as e:
        print(f"Error occurred for {link}: {e}")

# 브라우저 종료
driver.quit()

# 데이터 프레임으로 변환
result_df = pd.DataFrame(data)


Processing links: 16it [00:14,  1.07it/s]


In [39]:
create_sql = """
    CREATE TABLE IF NOT EXISTS Articles_detail (
        A_id INT AUTO_INCREMENT PRIMARY KEY,
        P_id INT,
        link VARCHAR(100),
        meta_data TEXT,
        text TEXT,
        images TEXT,
        FOREIGN KEY (P_id) REFERENCES Articles(id)   
    )
"""
upload_sql = """
    INSERT INTO articles_detail (P_id, link, meta_data, text, images)
    VALUES (%s, %s, %s, %s, %s)
"""

Connector.create_table(create_sql)
Connector.upload_data(upload_sql, result_df)

connected
Table Created
connected
오류 : Not enough parameters for the SQL statement


#### 전체 코드

In [40]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import mysql.connector


# 특정 컬럼의 하이퍼링크 가져오기
links = df['link'].tolist()
data = []

# ChromeDriver 경로 설정
chrome_driver_path = './chromedriver-win64/chromedriver-win64/chromedriver.exe'  # 실제 chromedriver 경로로 바꾸세요

# 셀레니움 설정
options = Options()
options.add_argument('--headless')  # 브라우저 창을 표시하지 않도록 설정
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

def get_metadata(soup):
    meta_data = {meta.get('name'): meta.get('content') for meta in soup.find_all('meta') if meta.get('name')}
    return meta_data

def get_text(soup):
    paragraphs = soup.find_all('p')
    text = '\n'.join([para.get_text() for para in paragraphs])
    return text

def get_images(soup):
    images = [img['src'] for img in soup.find_all('img') if 'src' in img.attrs]
    return images

# tqdm을 사용하여 진행 상황 표시
for index, link in tqdm(enumerate(links), total=len(links), desc="Processing links"):
    try:
        response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
        else:
            driver.get(link)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
        
        meta_data = get_metadata(soup)
        text = get_text(soup)
        images = get_images(soup)
        
        data.append({
            'P_id': df['id'].iloc[index],  # df의 id를 p_id로 추가
            'link': link,
            'meta_data': json.dumps(meta_data),  # 딕셔너리를 JSON 문자열로 변환
            'text': text,
            'images': json.dumps(images)  # 이미지 리스트를 JSON 문자열로 변환
        })
    except Exception as e:
        print(f"Error occurred for {link}: {e}")

# 브라우저 종료
driver.quit()

# 데이터 프레임으로 변환
result_df = pd.DataFrame(data)

# 모든 데이터가 문자열인지 확인
result_df = result_df.astype(str)

# MySQL 업로드 메서드 정의
class Connector:
    @staticmethod
    def create_table(create_sql):
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        cursor.execute(create_sql)
        conn.commit()
        cursor.close()
        conn.close()

    @staticmethod
    def upload_data(sql, df):
        df = df.where(pd.notnull(df), "no data")  # 결측치를 "no data"로 대체

        try:
            # 데이터베이스 연결 생성
            conn = mysql.connector.connect(**db_config)
            if conn.is_connected():
                cursor = conn.cursor()
                print("connected")
                
                # SQL 쿼리 실행
                for _, row in df.iterrows():
                    cursor.execute(sql, tuple(row))
                conn.commit()
                print("Upload Complete..!")
            cursor.close()
            conn.close()
        except mysql.connector.Error as e:
            print(f"Error: {e}")
            if conn.is_connected():
                conn.rollback()
                cursor.close()
                conn.close()

# MySQL 데이터베이스 연결 설정
db_config = {
    "host": "localhost",
    "user": "root",  # 본인의 MySQL 사용자 이름으로 변경
    "password": "wjdtkfkd12",  # 본인의 MySQL 비밀번호로 변경
    "database": "jiggle",
    "port": 3306,
    "auth_plugin": "mysql_native_password"
}

# 테이블 생성 SQL
create_sql = """
CREATE TABLE IF NOT EXISTS Articles_detail (
    A_id INT AUTO_INCREMENT PRIMARY KEY,
    P_id INT,
    link VARCHAR(255),
    meta_data TEXT,
    text TEXT,
    images TEXT,
    FOREIGN KEY (P_id) REFERENCES Articles(id)
)
"""

# 데이터 삽입 SQL
upload_sql = """
    INSERT INTO Articles_detail (P_id, link, meta_data, text, images)
    VALUES (%s, %s, %s, %s, %s)
"""

# 테이블 생성
Connector.create_table(create_sql)

# 데이터 업로드
Connector.upload_data(upload_sql, result_df)


Processing links: 100%|██████████| 16/16 [00:16<00:00,  1.02s/it]


connected
Upload Complete..!
